# Module 

In [161]:
import spotipy 
import requests 
import pandas as pd
import ast
import configparser
from typing import List
from os import listdir
import spotipy.util as util
from datetime import datetime

# Config Eingaben, Pfad und Token

In [156]:
username = ''
config = configparser.ConfigParser()
# Config spezifizieren
config.read('config.cfg')
client_id = config.get('SPOTIFY', 'CLIENT_ID')
client_secret = config.get('SPOTIFY', 'CLIENT_SECRET')
scope = 'user-read-recently-played'
redirect_uri = 'http://localhost:7777/callback'

In [ ]:
token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

# Funktionen 

## Streamings ablesen 

In [165]:
def get_streamings(path: str = 'path') -> List[dict]:
    '''
    input = path der spotify-daten
    
    return = list mit allen Streamings
    
    '''
    files = ['Path' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming in new_streamings]
            
        for streaming in all_streamings:
            streaming['datetime'] = datetime.strptime(streaming['endTime'], '%Y-%m-%d %H:%M')
    return all_streamings

## Song id 

In [223]:
def get_id(track_name: str, token: str, artist: str):
    '''
    input: 
    track name,
    token (zugriffsberechtigung),
    artist (Künstlername)
    
    return = track_id
    
    '''
    
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
        ('q','{}&' and '&{}'.format(track_name, artist)),
        ('type', 'track'),
    ]
    
    response = requests.get('https://api.spotify.com/v1/search', 
                headers = headers, params = params, timeout = 5)
    json = response.json()
    first_result = json['tracks']['items'][0]
    track_id = first_result['id']
    return track_id

In [213]:
def get_features(track_id: str, token: str ) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [221]:
lucy_id = get_id('Mystery of Love', token, 'Sufjan Stevens')
print(lucy_id)

5GbVzc6Ex5LYlLJqzRQhuy


In [222]:
lucy_features = get_features(lucy_id, token)
print(lucy_features)

{'danceability': 0.365, 'energy': 0.273, 'key': 4, 'loudness': -16.526, 'mode': 0, 'speechiness': 0.038, 'acousticness': 0.94, 'instrumentalness': 0.431, 'liveness': 0.109, 'valence': 0.238, 'tempo': 132.285, 'type': 'audio_features', 'id': '5GbVzc6Ex5LYlLJqzRQhuy', 'uri': 'spotify:track:5GbVzc6Ex5LYlLJqzRQhuy', 'track_href': 'https://api.spotify.com/v1/tracks/5GbVzc6Ex5LYlLJqzRQhuy', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5GbVzc6Ex5LYlLJqzRQhuy', 'duration_ms': 248960, 'time_signature': 5}


In [ ]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName'] 
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})